# Data Analysis (Flights)

A real-world dataset containing flights data from the US Department of Transportation is explored in this tutorial.

We start with loading the data.

In [10]:
import pandas as pd

flights = pd.read_csv('./datasets/flights.csv')

print('The data set contains {} rows and {} columns.'.format(flights.shape[0], flights.shape[1]))

The data set contains 271940 rows and 20 columns.


There seems 271940 flights appeared in the data set. Let us have look at the first rows.

In [11]:
flights.head()

,Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,OriginAirportName,OriginCity,OriginState,DestAirportID,DestAirportName,DestCity,DestState,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
0,2013,9,16,1,DL,15304,Tampa International,Tampa,FL,12478,John F. Kennedy International,New York,NY,1539,4,0.0,1824,13,0,0
1,2013,9,23,1,WN,14122,Pittsburgh International,Pittsburgh,PA,13232,Chicago Midway International,Chicago,IL,710,3,0.0,740,22,1,0
2,2013,9,7,6,AS,14747,Seattle/Tacoma International,Seattle,WA,11278,Ronald Reagan Washington National,Washington,DC,810,-3,0.0,1614,-7,0,0
3,2013,7,22,1,OO,13930,Chicago O'Hare International,Chicago,IL,11042,Cleveland-Hopkins International,Cleveland,OH,804,35,1.0,1027,33,1,0
4,2013,5,16,4,DL,13931,Norfolk International,Norfolk,VA,10397,Hartsfield-Jackson Atlanta International,Atlanta,GA,545,-1,0.0,728,-9,0,0


We can take a close look at the columns.

In [12]:
flights.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier',
       'OriginAirportID', 'OriginAirportName', 'OriginCity', 'OriginState',
       'DestAirportID', 'DestAirportName', 'DestCity', 'DestState',
       'CRSDepTime', 'DepDelay', 'DepDel15', 'CRSArrTime', 'ArrDelay',
       'ArrDel15', 'Cancelled'],
      dtype='object')

The data set has $20$ columns:

- Year: Should always be $2013$ for this data set 
- Month: The month of the flight
- DayofMonth: The day of the flight
- DayOfWeek: The week day of the flight
- Carrier: The carrier of the flight
- OriginAirportID: The origin airport id
- OriginAirportName: The origin airport name
- OriginCity: The origin city 
- OriginState: The origin state
- DestAirportID: The destination airport id
- DestAirportName: The destination airport name
- DestCity: The destination city
- DestState: The destination state
- CRSDepTime: The scheduled departure time
- DepDelay: Delay in the departure time (mins)
- DepDel15: Whether the flight is delayed at the departure more than 15 mins (binary)
- CRSArrTime: The scheduled arrival time
- ArrDelay: Delay in the arrival time (mins)
- ArrDel15: Whether the flight is delayed at the arrival more than 15 mins (binary)
- Cancelled: Whether the flight is cancelled (binary)

## Data cleaning

Let us have a look at whether the data set contains any null or missing data. 

In [13]:
flights.isnull().sum()

Year                    0
Month                   0
DayofMonth              0
DayOfWeek               0
Carrier                 0
OriginAirportID         0
OriginAirportName       0
OriginCity              0
OriginState             0
DestAirportID           0
DestAirportName         0
DestCity                0
DestState               0
CRSDepTime              0
DepDelay                0
DepDel15             2761
CRSArrTime              0
ArrDelay                0
ArrDel15                0
Cancelled               0
dtype: int64

It seems that the column `DepDel15` includes $2761$ null values. As this columns is directly computed using `DepDelay` column, it is wise to first check the corresponding lines.

In [14]:
flights[flights.isnull().any(axis=1)].DepDelay.describe()

count    2761.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: DepDelay, dtype: float64

As seen above, they are all $0$ values for the departures. It seems suspicious. They cannot be all zero minutes. Let us have a look at the whole column description.

In [15]:
flights.DepDelay.describe()

count    271940.000000
mean         10.350449
std          35.673710
min         -63.000000
25%          -4.000000
50%          -1.000000
75%           9.000000
max        1425.000000
Name: DepDelay, dtype: float64

The mean value and the median value show us that an average flight is not delayed more than 15 minutes. Thus, we can safely replace null values by $0$.

In [16]:
flights.DepDel15 = flights.DepDel15.fillna(0)

There exists no null values in our data set. Now, we can move on to the exploring the data in detail. We can have a look at the outliars. Recall our columns, it is not wise to check all the columns. Actually, an outlier can exist on only two columns, namely `DepDelay` and `ArrDelay`.